# Agent Based Staircase Model

## This notebook implements an agent-based approach to modeling the travel of people up and down a staircase simultaneously. 

Each agent has the following characteristics:
- Age (years)
- Sex (M/F)
- Height (cm)
- Weight (kg)
- Group
- Stepping Pattern (traditional: one footstep per stair, double: two footsteps per stair, skip: only step on every other stair)
    - Sampled from a distribution dependent on the age, height, and weight of the agent
- Intended Final Destination
- Desired Velocity $v$ (m/s)
- Ascending/Descending

The following parameters are used to define the staircase domain:
- The number of stairs $N$
- The length of each step ${l}_{i = 1}^N $
- The width of each step ${w}_{i=1}^N $

It is assumed that the steps are continuous in that the length of the far edge must align with the length of the close edge of the next step and vice versa.

Groups arrive at the staircase according to a Poisson Arrival Process. When a group arrives, the size of the group is sampled from a distribution and each of the members of the group are introduced into the model. 

The initial locations of the agents within the staircase are found by first introducing them in a horizonal arrangement (i.e. shoulder-to-shoulder) in an open space before the staircase subject to the same attracion and repulsions which govern movement within the staircase. This open space then narrows to match the dimensions of the fist stair, serving as a funnel. This enables the groups to form an inital arrangement on the staircase which resembles a natural group walking formation.

Attractive Forces:
- Desire to reach intended destination atop the staircase by following a linear trajectory (most efficient route between points a and b is a straight line)
- Desire to maintain intended velocity
- Desire to maintain a certain distance between group members
- Desire to obey agreed upon social convention (walking on the right side vs left side)

Repulsive Forces:
- Desire not to get too close to other people 
    - the strength of this desire depends on whether the other agent is a member of one's group and the direction in which they are traversing (e.g. One would have a stronger desire to avoid contact with someone going in the opposite direction)

These attractive and repulsive forces are then combined into a utility function which determines where the agent will step on the next stair in the $x$ dimension (along the width of the stair). 

The final output of this model is a dataset of simulated steps on each stair and their associated agent. These steps can easily be converted to represent the cumulative pressure placed on each stair over the time interval in which the simulation is run. With a long enough sample, we can then extrapolate to a larger time interval by assuming periodicity in the stepping patterns. 


In [3]:
# Imports
import numpy as np

# Group Initialization
 - A Poisson Arrival Process dictates when the groups arrive ($\lambda$ is a model parameter)
 - Group size is sample from a right-skewed distribution (this can be varried as a model parameter)
 - Agents are then created by placing the group in a horizontal formation inside the funnel either at the top or bottom of the staircase

## Agent Class

In [ ]:
class StairAgent(Agent):
    def __init__(self, unique_id, model, age, sex, height, weight, stepping_pattern, velocity, direction, group_id):
        super().__init__(unique_id, model)
        self.age = age
        self.sex = sex
        self.height = height
        self.weight = weight
        self.stepping_pattern = stepping_pattern
        self.velocity = velocity
        self.direction = direction
        self.group_id = group_id
        self.position = None  # To be initialized in open space 

    def step(self):
        # Compute forces and update position
        forces = self.calculate_forces()
        self.position = self.update_position(forces)

    
    def calculate_forces(self):
        # Calculate attractive and repulsive forces
        # Example: destination force, velocity maintenance
        return force 

    def update_position(self, forces):
        # Update position based on forces
        new_x = self.position[0] + forces["x_force"]
        new_y = self.position[1] + forces["y_force"]
        return (new_x, new_y)

## Domain

Create the staircase domain

## Model

In [ ]:
from mesa import Agent, Model
from mesa.space import ContinuousSpace
from mesa.time import SimultaneousActivation
import numpy as np

# Define the Agent
class StairAgent(Agent):
    def __init__(self, unique_id, model, age, sex, height, weight, stepping_pattern, velocity, direction, group_id):
        super().__init__(unique_id, model)
        self.age = age
        self.sex = sex
        self.height = height
        self.weight = weight
        self.stepping_pattern = stepping_pattern
        self.velocity = velocity
        self.direction = direction
        self.group_id = group_id
        self.position = None  # To be initialized in the open space

    def step(self):
        # Compute forces and update position
        forces = self.calculate_forces()
        self.position = self.update_position(forces)

    def calculate_forces(self):
        # Calculate attractive and repulsive forces
        # Example: destination force, velocity maintenance
        return {"x_force": 0, "y_force": 0}

    def update_position(self, forces):
        # Update position based on forces
        new_x = self.position[0] + forces["x_force"]
        new_y = self.position[1] + forces["y_force"]
        return (new_x, new_y)

# Define the Model
class StaircaseModel(Model):
    def __init__(self, num_agents, num_stairs, stair_length, stair_width):
        self.num_agents = num_agents
        self.num_stairs = num_stairs
        self.stair_length = stair_length
        self.stair_width = stair_width
        self.schedule = SimultaneousActivation(self)
        self.space = ContinuousSpace(stair_length, stair_width, True)

        # Initialize agents
        for i in range(num_agents):
            age = np.random.randint(10, 70)  # Example age distribution
            sex = np.random.choice(["M", "F"])
            height = np.random.normal(170, 10)
            weight = np.random.normal(70, 15)
            stepping_pattern = np.random.choice(["traditional", "double", "skip"])
            velocity = np.random.uniform(0.5, 1.5)
            direction = np.random.choice(["ascending", "descending"])
            group_id = np.random.randint(0, 10)
            agent = StairAgent(i, self, age, sex, height, weight, stepping_pattern, velocity, direction, group_id)
            self.schedule.add(agent)
            agent.position = self.space.place_agent(agent, (0, i * 2))  # Example initialization

    def step(self):
        self.schedule.step()

# Run the model
model = StaircaseModel(num_agents=100, num_stairs=10, stair_length=1.0, stair_width=1.0)
for i in range(100):
    model.step()
